In [1]:
import sys
import pandas as pd
import geopandas as gpd
import datetime
import numpy as np
from geopy.distance import geodesic
import matplotlib.pyplot as plt
import scipy
from sklearn.metrics import r2_score

In [2]:
def read_df_from_file(filename):
	df = pd.read_csv(filename, sep=";")
	return df

In [3]:
def read_shapefile_region(shapefile_name,city_list):
	# Essa função apenas lê o shapefile e filtra os municípios de interesse.
	# Retorna um geodataframe para ser tratado depois.

	gdf_all = gpd.read_file(shapefile_name)#, layer='countries')
	
	gdf_region = gdf_all[gdf_all.NM_MUNICIP.isin(city_list)]
	
	#print(gdf_all.head(10))
	#print(gdf_region)
	#print(len(gdf_region.index))
	
	return [gdf_region]

In [4]:
def calculate_closest_antenna(gdf_region,df_antennas):
	# Essa função lê o gdf_region e o df_antennas e calcula qual é a antena mais 
	# próxima de cada região.
	# Esse esquema vai mudar completamente com o Voronoi.	
	
	# Aqui eu vou:
	#	+ criar uma lista de setores (com cada linha do gdf_region)
	#	+ obter lat e long de cada setor
	#	+ percorrer o df_antennas e ver qual a antena mais próxima do setor
	#	+ cada linha do setor/closest_antenna vai ter id do setor/id da antena mais próxima
	
	setores = []
	closest_antennas = []
	distance_antennas = []
	
	#"""
	for setor in gdf_region.itertuples():
		
		
		setor_long = setor.geometry.centroid.x
		setor_lat = setor.geometry.centroid.y
		#print(setor.ID, setor.CD_GEOCODI, setor_lat, setor_long)
		least_distance = np.Inf
		least_distance_same_city = np.Inf
		
		
		for antenna in df_antennas.itertuples():

			antenna_lat = antenna.LAT
			antenna_long = antenna.LONG
			
			#[distance] = calculate_distance_two_coordinates(antenna_lat,antenna_long, setor_lat, setor_long)
			#print(antenna_lat, antenna_long, setor_lat, setor_long, distance)
			
			coords_1 = (antenna_lat, antenna_long)
			coords_2 = (setor_lat, setor_long)
			distance = geodesic(coords_1, coords_2).kilometers
			#print("distância para a antena", antenna.antenna, " = ",distance)
			
			if antenna.CITY == setor.NM_MUNICIP:
				if distance < least_distance_same_city:
					least_distance_same_city = distance
					closest_antenna_same_city = antenna.antenna
			
			if distance < least_distance:
				#print("atualizou a distância")
				least_distance = distance
				closest_antenna = antenna.CELLID
				
		#end for antenna
		
			
		#print("\n\n\t\tantena mais próxima do setor",setor.CD_GEOCODI,":", closest_antenna, "(dist = ",least_distance,")")
		
		setores.append(setor.CD_GEOCODI)
		if least_distance_same_city != np.Inf:
			closest_antennas.append(closest_antenna_same_city)
			distance_antennas.append(least_distance_same_city)
		else:
			closest_antennas.append(closest_antenna)
			distance_antennas.append(least_distance)
	#end for setor
	#"""
		
	return [setores,closest_antennas,distance_antennas]
#end

In [5]:
city_list = ['SÃO JOÃO DEL REI', 'TIRADENTES', 'SÃO VICENTE DE MINAS', 'SÃO TIAGO', 'SANTA CRUZ DE MINAS', 'RITÁPOLIS', 'RESENDE COSTA', 'PRADOS', 'PIEDADE DO RIO GRANDE', 'NAZARENO', 'CORONEL XAVIER CHAVES', 'CONCEIÇÃO DA BARRA DE MINAS', 'MADRE DE DEUS DE MINAS', 'LAGOA DOURADA']
[gdf_region] = read_shapefile_region("/home/gustavo/Desktop/Mestrado/mestrado_dados/Arquivos_Shape/mg_setores_censitarios/31SEE250GC_SIR.shp",city_list)


In [6]:
df_residence_antennas = pd.read_csv("/home/gustavo/Desktop/Mestrado/mestrado_dados/Regiao_Presumida/residence_antenna.csv", sep=";")
df_antennas = pd.read_csv("/home/gustavo/Desktop/Mestrado/mestrado_dados/ProcessamentoVinicius/Sjdr/antennas_sjdr_unique.txt", sep=";")


In [7]:
[setores,closest_antennas,distance_antennas] = calculate_closest_antenna(gdf_region,df_antennas)

In [10]:
d = {'setor': setores, 'closest_antenna': closest_antennas, 'distance_antenna': distance_antennas}
df_info_setores_antennas = pd.DataFrame(data=d)
df_info_setores_antennas.to_csv (r'/home/gustavo/Desktop/Mestrado/mestrado_dados/ibge/info_setores_antennas_sjdr.csv', index = False, header=True, sep=";")